In [1]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
import pandas as pd
from tqdm import tqdm

# Preparing the OD Matrices

## 1. Splitting the OD Matrix

In [2]:
od_matrix = pd.read_csv('../data/d02_processed-safegraph/weeks_od_us_fua.csv')

First we will split the od_matrix into multiple dataframes based on the FUA area and save each of them:

In [3]:
od_matrix_dict = {fua: df for fua, df in od_matrix.groupby('fuacode')}

In [ ]:
for fua_code, df in od_matrix_dict.items():
    df.to_csv('../data/d02_processed-safegraph/OD-per-FUA/' + fua_code+'_ODmatrix.csv')

## 2. Get all the Safegraph Represented FUAs

We will get a list of all FUA codes that we need to work with for bookkeeping and for running the bash scripts from arrays.

In [4]:
fua_codes = list(set(od_matrix.fuacode.dropna().values))
def num(code):
    return int(code[3:])
fua_codes.sort(key=num)

Save the numbers so we do not have to open the big dataframe again and can use in the sbatch array. Experimentally I observed that later-numbered FUAs seem to be smaller. So it makes sense to put those first:

In [5]:
outfile = open('../data/d02_processed-safegraph/safegraph_fua.txt', 'w')
print('\n'.join(str(i) for i in fua_codes[::-1]), file=outfile)
outfile.close()

## 3. Getting Preferred Network and Nearest Nodes

The function that does the job:

In [6]:
def get_expanded_OD(fua_code, threshold=2000):
    print('FUA: ', fua_code)
    if fua_code not in ['USA02', 'USA56']:
        #Get the files:
        od_matrix = pd.read_csv('../data/d02_processed-safegraph/OD-per-FUA/'+fua_code+'_ODmatrix.csv').drop(['Unnamed: 0'], axis=1)
        walk_graph = ox.project_graph(ox.load_graphml('../data/d03_intermediate/FUA-networks/walk/'+fua_code+'.graphml'), to_crs='EPSG:5070')
        drive_graph = ox.project_graph(ox.load_graphml('../data/d03_intermediate/FUA-networks/drive/'+fua_code+'.graphml'), to_crs='EPSG:5070')
        print('  got all files')
        
        #Get the geometries of origin and destinations:
        places_pt = gpd.points_from_xy(x= od_matrix.longitude, y=od_matrix.latitude, crs='EPSG:4326').to_crs('EPSG:5070')
        centroids_pt = gpd.points_from_xy(x= od_matrix.intptlon, y=od_matrix.intptlat, crs='EPSG:4326').to_crs('EPSG:5070')

        od_matrix['origin_x'], od_matrix['origin_y'] = centroids_pt.x, centroids_pt.y
        od_matrix['dest_x'], od_matrix['dest_y'] = places_pt.x, places_pt.y

        #Get the Boolean value of whether we walk or drive:
        od_matrix['walk'] = places_pt.distance(centroids_pt) <= threshold
        print('  got preferred mode')

        #Now we split the dataframe into two (one for walking and one for driving):
        od_matrix_dict = {walk: df for walk, df in od_matrix.groupby('walk')}
        G = {False: drive_graph, True: walk_graph}

        #For each of those dataframes, we do nearest nodes from OSMnx on the appropriate graph:
        full_dfs = []
        for walk, df in od_matrix_dict.items():
            df['origin_node'] = ox.nearest_nodes(G[walk], df['origin_x'], df['origin_y'])
            df['destination_node'] = ox.nearest_nodes(G[walk], df['dest_x'], df['dest_y'])
            #HERE WE CAN GET THE DISTANCE AS WELL! This step is further detailed in the script
            full_dfs.append(df)

        merged_df = pd.concat(full_dfs, ignore_index=True)
        expanded_OD = merged_df.drop(['origin_x', 'origin_y', 'dest_x', 'dest_y'], axis=1)
        print('  got expanded matrix')
        
        expanded_OD.to_csv('../data/d04_final-OD-matrices/OD-per-FUA_nodistance/' + fua_code+'_expanded_nodistance-ODmatrix.csv')
        print('  saved')
    
    else:
        expanded_OD = None
        
    return expanded_OD

This will be ran in the cluster with multiple cores for the osmnx distance function

In [ ]:
for fua in tqdm(fua_codes):
    expanded_OD = get_expanded_OD(fua)

Note that we could first run this script to get all the extended matrices (with closest nodes and mode of commute but no distances). This however would save us little time: those operations take around 1-3 minutes only for each FUA.